Extraindo e limapndo dados

In [ ]:
import pandas as pd 

from sklearn.model_selection import KFold

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import (train_test_split,
                                     cross_validate,
                                     KFold, StratifiedKFold)

from sklearn.compose import make_column_transformer



url = '/home/fagner/ProjetoAlura3/data/TelecomX_Data_com_0_e_1.csv'
url2 = '/home/fagner/ProjetoAlura3/data/TelecomX_Data_Normalizado.csv'
df = pd.read_csv(url)
df2 = pd.read_csv(url2)

print(df.info())

df_clean = df.drop(['customerid','churn'] ,axis=1)
Y = df['churn']
df['internetservice'] = df2['internetservice']
df['contract'] = df2['contract']
df['paymentmethod'] = df2['paymentmethod']


colums = ['internetservice','contract','paymentmethod']
colunas = df_clean.columns.tolist()

one_hot = one_hot = make_column_transformer((OneHotEncoder(drop='if_binary'),colums),remainder='passthrough',sparse_threshold=0)
df_n = one_hot.fit_transform(df_clean)
new_df = pd.DataFrame(df_n ,columns=one_hot.get_feature_names_out(colunas) )


#new_df.to_csv('/home/fagner/ProjetoAlura3/data/TelecomX_Data_Normalizado_limpo.csv')
#organizando dados 




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7267 non-null   object 
 1   churn             7267 non-null   int64  
 2   internetservice   7267 non-null   int64  
 3   onlinesecurity    7267 non-null   int64  
 4   onlinebackup      7267 non-null   int64  
 5   deviceprotection  7267 non-null   int64  
 6   techsupport       7267 non-null   int64  
 7   streamingtv       7267 non-null   int64  
 8   streamingmovies   7267 non-null   int64  
 9   contract          7267 non-null   int64  
 10  paperlessbilling  7267 non-null   int64  
 11  paymentmethod     7267 non-null   int64  
 12  charges_monthly   7267 non-null   float64
 13  charges_total     7256 non-null   float64
 14  gender            7267 non-null   int64  
 15  seniorcitizen     7267 non-null   int64  
 16  partner           7267 non-null   int64  
